In [1]:
# !pip install evaluate
# import sys
# sys.path.append('/kaggle/input/qwertttt')

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 

import torch 

import transformers 
from transformers import BertTokenizer 
from typing import * 

from MEOW_Models.MT_models import MEOW_MTM 
from MEOW_Models.Kernel_model import BertWithoutEmbedding 

from MEOW_Utils.Data_utils import* 
from MEOW_Utils.Training_utils import* 
from MEOW_Utils.config import * 

In [2]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODULE_NAME)
## use it if you only want to change the datasize of some dataset(dataframe)
# create_CoLA_df(ORG_FILE_PATH_CoLA, tokenizer, data_size=CoLA_DATASIZE)
# create_MNLI_df(ORG_FILE_PATH_MNLI, tokenizer, data_size=MNLI_DATASIZE)
# create_SQuAD_df(ORG_FILE_PATH_SQuAD, tokenizer, data_size=SQuAD_DATASIZE)
# create_QNLI_df(ORG_FILE_PATH_QNLI, tokenizer, data_size=QNLI_DATASIZE)

c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\Data_utils.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['TKstart'][i], df_train['TKend'][i] = count_the_TKbeg_and_TKend(df_train.iloc[i]['context'], df_train.iloc[i]['answer_start'], df_train.iloc[i]['text'], tokenizer)


,index,question,context,text,answer_start,label,TKstart,TKend,SEP_ind
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269,1,67,70,165
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207,1,55,57,165
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526,1,128,128,165
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"houston, texas",166,1,47,49,165
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276,1,69,70,165
...,...,...,...,...,...,...,...,...,...
99840,5acd74fd07355d001abf4325,Why wasn't secondary school introduced in the...,Secondary education in the United States did n...,,-1,0,0,0,94
99841,5acd74fd07355d001abf4326,Who didn't benefit from secondary schools?,Secondary education in the United States did n...,,-1,0,0,0,94
99842,5acd74fd07355d001abf4327,Why were high schools not created?,Secondary education in the United States did n...,,-1,0,0,0,94
99843,572b8770111d821400f38ee8,What was another name used for Higher Education?,"Higher education, also called tertiary, third ...","tertiary, third stage, or postsecondary education",30,1,6,16,95


In [3]:
#處理好 dataframe
df_CoLA = pd.read_csv(INPUT_FILE_PATH_CoLA, index_col=[0])
df_MNLI = pd.read_csv(INPUT_FILE_PATH_MNLI, index_col=[0])
df_SQuAD = pd.read_csv(INPUT_FILE_PATH_SQuAD, index_col=[0])
df_QNLI = pd.read_csv(INPUT_FILE_PATH_QNLI, index_col=[0])

df_SQuAD_HA = df_SQuAD[df_SQuAD.answer_start != -1]
df_SQuAD_NA = df_SQuAD[df_SQuAD.answer_start == -1]

# df_SQuAD.sample(n=8)

In [4]:
Helper = layer_helper(pretrained_module_name=PRETRAINED_MODULE_NAME, device=DEVICE)

SQuAD_NA_databox = DataBox(
                 dataset_name = 'SQuAD',
                 embedding_layer = Helper.get_embedding_layer(individual=False),
                 clfmodeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
                 df_Data = df_SQuAD_NA,
                 test_size = TEST_SIZE,
                 tokenizer = tokenizer,
                 label_nums = 2,
                 batch_size = Target_BATCH_SIZE,
                 )

SQuAD_HA_databox = DataBox(
                 dataset_name = 'SQuAD',
                 embedding_layer = SQuAD_NA_databox.embedding_layer,
                 clfmodeling_layer = SQuAD_NA_databox.clf_modeling_layer,
                 df_Data = df_SQuAD_HA,
                 test_size = TEST_SIZE,
                 tokenizer = tokenizer,
                 label_nums = 2,
                 batch_size = Target_BATCH_SIZE,
                 )

CoLA_databox = DataBox(
             dataset_name = 'CoLA',
             embedding_layer = Helper.get_embedding_layer(individual=False),
             clfmodeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
             df_Data = df_CoLA,
             test_size = TEST_SIZE,
             tokenizer = tokenizer,
             label_nums = 2,
             batch_size = Support_BATCH_SIZE
             )

MNLI_databox = DataBox(
             dataset_name = 'MNLI',
             embedding_layer = Helper.get_embedding_layer(individual=False),
             clfmodeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
             df_Data = df_MNLI,
             test_size = TEST_SIZE,
             tokenizer = tokenizer,
             label_nums = 3,
             batch_size = Support_BATCH_SIZE
             )

QNLI_databox = DataBox(
             dataset_name = 'QNLI',
             embedding_layer = Helper.get_embedding_layer(individual=False),
             clfmodeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
             df_Data = df_QNLI,
             test_size = TEST_SIZE,
             tokenizer = tokenizer,
             label_nums = 2,
             batch_size = Support_BATCH_SIZE
             )

support_dataBox_list = [CoLA_databox, MNLI_databox, QNLI_databox]


# for i in SQuAD_NA_databox.test_dataloader:
#     0
# for i in SQuAD_HA_databox.test_dataloader:
#     0
# for i in MNLI_databox.test_dataloader:
#     0
# for i in CoLA_databox.test_dataloader:
#     0
for i in QNLI_databox.test_dataloader:
    0

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithoutEmbedding: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertWithoutEmbedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithoutEmbedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
MEOW_model = MEOW_MTM(kernel_model=Helper.get_kernel_model(),
                      modeling_layer_for_qa=Helper.get_modelings_layer_qa(),
                      qa_databox = SQuAD_HA_databox,  #pass HA or NA is same
                      support_databox_list = support_dataBox_list,
                      device=DEVICE
                      ) 
MEOW_model.to(DEVICE)

# H = {
#     "train_loss": [],
#     "train_acc": [],
#     "val_loss":[],
#     "val_acc": []
#     }

Training_round = min(len(SQuAD_HA_databox.training_dataloader),
                     len(SQuAD_NA_databox.training_dataloader),
                     len(MNLI_databox.training_dataloader),
                     len(CoLA_databox.training_dataloader),
                     len(QNLI_databox.training_dataloader))

Test_round = min(len(SQuAD_HA_databox.test_dataloader),
                 len(SQuAD_NA_databox.test_dataloader),
                 len(MNLI_databox.test_dataloader),
                 len(CoLA_databox.test_dataloader),
                 len(QNLI_databox.test_dataloader))

print(f'Per epoc round\'s num is {Training_round}')
print(f'Per epoc round\'s num is {Test_round}')

Per epoc round's num is 978
Per epoc round's num is 420


In [ ]:
df = pd.concat([SQuAD_HA_databox.df_test, SQuAD_NA_databox.df_test]).reset_index(drop=True)
count_F1_score(MEOW_model, df, tokenizer, DEVICE)

In [6]:
# 訓練
for epoch in range(EPOCH_NUM):
    print("the {:d} iter :".format(epoch+1))

    HA_record = Record()
    NA_record = Record()
    Sup_record = [Record() for _ in range(SUPPORT_DATA_NUM)]

    #### training now 
    ################################################################################################################
    MEOW_model.train()

    support_data_training_iter = [iter(db.training_dataloader) for db in support_dataBox_list]
    SQuAD_HA_training_iter = iter(SQuAD_HA_databox.training_dataloader)
    SQuAD_NA_training_iter = iter(SQuAD_NA_databox.training_dataloader)

    for _ in range(Training_round):
        # SQuAD has_answer
        loss, prob, correct = QA_running(MEOW_model, SQuAD_HA_training_iter, DEVICE, dataset_ind = DATA_IND['SQuAD'], do_optimize=True)
        HA_record.add_training_acur(correct, prob.size(0))
        HA_record.add_training_loss(loss.item())

        # SQuAD no_answer
        loss, prob, correct = QA_running(MEOW_model, SQuAD_NA_training_iter, DEVICE, dataset_ind = DATA_IND['SQuAD'], do_optimize=True)
        NA_record.add_training_acur(correct, prob.size(0))
        NA_record.add_training_loss(loss.item())

        #### support data train -------------------------------------
        for i in range(SUPPORT_DATA_NUM):
            loss, prob, correct = Classifiaction_running(MEOW_model, support_data_training_iter[i], DEVICE, i, do_optimize=True)
            Sup_record[i].add_training_acur(correct, prob.size(0))
            Sup_record[i].add_training_loss(loss.item())
        #### --------------------------------------------------------
    
    #### record ############################
    SQuAD_HA_databox.H['train_loss'].append(HA_record.get_training_average_loss())
    SQuAD_HA_databox.H['train_acc'].append(HA_record.get_training_accuracy())
    print("SQuAD has answer train loss: {:.6f}, correct_rate: {:.4f}".format(HA_record.get_training_average_loss(), HA_record.get_training_accuracy()))

    SQuAD_NA_databox.H['train_loss'].append(NA_record.get_training_average_loss()) 
    SQuAD_NA_databox.H['train_acc'].append(NA_record.get_training_accuracy())
    print("SQuAD no answer train loss: {:.6f}, correct_rate: {:.4f}".format(NA_record.get_training_average_loss(), NA_record.get_training_accuracy()))
    
    for i in range(SUPPORT_DATA_NUM):
        x = Sup_record[i].get_training_average_loss()
        y = Sup_record[i].get_training_accuracy()
        support_dataBox_list[i].H['train_loss'].append(x)
        support_dataBox_list[i].H['train_acc'].append(y)
        print("{} train loss: {:.6f}, correct_rate: {:.4f}".format(DATA_NAME[i], x, y))
    ########################################

    ################################################################################################################


    #### test now
    ################################################################################################################
    MEOW_model.eval()

    support_data_test_iter = [iter(db.test_dataloader) for db in support_dataBox_list]
    SQuAD_HA_test_iter = iter(SQuAD_HA_databox.test_dataloader)
    SQuAD_NA_test_iter = iter(SQuAD_NA_databox.test_dataloader)

    for _ in range(Test_round):
        # SQuAD has_answer
        loss, prob, correct = QA_running(MEOW_model, SQuAD_HA_test_iter, DEVICE, dataset_ind = DATA_IND['SQuAD'], do_optimize=False)
        HA_record.add_test_acur(correct, prob.size(0))
        HA_record.add_test_loss(loss.item())

        # SQuAD no_answer
        loss, prob, correct = QA_running(MEOW_model, SQuAD_NA_test_iter, DEVICE, dataset_ind = DATA_IND['SQuAD'], do_optimize=False)
        NA_record.add_test_acur(correct, prob.size(0))
        NA_record.add_test_loss(loss.item())
        
        # Sup data 
        for i in range(SUPPORT_DATA_NUM):
            loss, prob, correct = Classifiaction_running(MEOW_model, support_data_test_iter[i], DEVICE, i, do_optimize=False)
            Sup_record[i].add_test_acur(correct, prob.size(0))
            Sup_record[i].add_test_loss(loss.item())
        
    #### record ################################
    SQuAD_HA_databox.H['test_loss'].append(HA_record.get_test_average_loss())
    SQuAD_HA_databox.H['test_acc'].append(HA_record.get_test_accuracy())
    print("SQuAD has answer test loss: {:.6f}, correct_rate: {:.4f}".format(HA_record.get_test_average_loss(), HA_record.get_test_accuracy()))

    SQuAD_NA_databox.H['test_loss'].append(NA_record.get_test_average_loss()) 
    SQuAD_NA_databox.H['test_acc'].append(NA_record.get_test_accuracy())
    print("SQuAD no answer test loss: {:.6f}, correct_rate: {:.4f}".format(NA_record.get_test_average_loss(), NA_record.get_test_accuracy()))
    
    for i in range(SUPPORT_DATA_NUM):
        x = Sup_record[i].get_test_average_loss()
        y = Sup_record[i].get_test_accuracy()
        support_dataBox_list[i].H['test_loss'].append(x)
        support_dataBox_list[i].H['test_acc'].append(y)
        print("{} test loss: {:.6f}, correct_rate: {:.4f}".format(DATA_NAME[i], x, y))
    #############################################
    print(" ")
    ################################################################################################################

the 1 iter :
the SQuAD has answer loss is :  12.614734649658203


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.28 GiB already allocated; 0 bytes free; 3.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch

a = torch.tensor([[[4,5,6]], [[1,1,1]]], dtype=float)  # 2 * 1 * 3
b = torch.tensor([  [[1,2,1],[1,2,1],[1,1,2]], [[0,2,1],[7,7,7],[3,3,3]]  ], dtype=float) # 2 * 3 * 3

mtx = torch.matmul(a,b).squeeze()
mtx

tensor([[15., 24., 21.],
        [10., 12., 11.]], dtype=torch.float64)

In [ ]:
a = torch.tensor([[4,5,6], [1,1,1]], dtype=float)  # 2 * 3
b = torch.tensor([  [[1,2,1],[1,2,1],[1,1,2]], [[0,2,1],[7,7,7],[3,3,3]]  ], dtype=float) # 2 * 3 * 3

mtx = torch.stack( [torch.matmul(a[i], b[i]) for i in range(2) ])
mtx

In [ ]:
a = torch.tensor([1,2,3])

b = a[None, :]

print(b.shape)

torch.Size([1, 3])
